# Coconut Leaf Disease Detection Model v1

## Coconut Health Monitor - Research Project

**Objective:** Train a model to detect leaf diseases in coconut trees.

### Model Details
- **Architecture:** EfficientNetB0 (Transfer Learning)
- **Classes:** 3 (healthy, Leaf Rot, Leaf_Spot)
- **Input Size:** 224x224x3
- **Loss Function:** Focal Loss (for class imbalance)

### Disease Detection Categories
1. **Healthy** - Normal healthy coconut leaves
2. **Leaf Rot** - Leaves affected by rot disease
3. **Leaf Spot** - Leaves with spot disease

### Anti-Overfitting Measures
1. Data augmentation (training only)
2. Dropout layers
3. Early stopping
4. Learning rate reduction
5. Class weights for imbalanced data
6. 2-phase training (frozen base → fine-tuning)

---

## 1. Setup and Imports

In [ ]:
# Standard Libraries
import os
import json
import shutil
import random
import warnings
from datetime import datetime
from pathlib import Path

# Data Processing
import numpy as np
import pandas as pd
from PIL import Image

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

# Metrics
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    precision_recall_fscore_support,
    accuracy_score
)
from sklearn.utils.class_weight import compute_class_weight

# Suppress warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Configuration

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

# Model Configuration
MODEL_NAME = "leaf_disease_detection_v1"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 3
CLASS_NAMES = ['Leaf Rot', 'Leaf_Spot', 'healthy']  # Alphabetical order

# Training Configuration
PHASE1_EPOCHS = 25  # Frozen base layers
PHASE2_EPOCHS = 35  # Fine-tuning
PHASE1_LR = 1e-3
PHASE2_LR = 1e-5

# Focal Loss Parameters
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25

# Paths
BASE_PATH = Path(r"D:\SLIIT\Reaserch Project\CoconutHealthMonitor\Research\ml")
DATA_PATH = BASE_PATH / "data" / "raw" / "stage_2_split"
MODEL_SAVE_PATH = BASE_PATH / "models" / MODEL_NAME

# Create output directory
MODEL_SAVE_PATH.mkdir(parents=True, exist_ok=True)

print("Configuration:")
print(f"  Model Name: {MODEL_NAME}")
print(f"  Image Size: {IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Classes: {CLASS_NAMES}")
print(f"  Phase 1 Epochs: {PHASE1_EPOCHS}")
print(f"  Phase 2 Epochs: {PHASE2_EPOCHS}")
print(f"\nData Path: {DATA_PATH}")
print(f"Model Save Path: {MODEL_SAVE_PATH}")

## 3. Dataset Analysis

In [ ]:
def analyze_dataset(data_path):
    """Analyze the dataset and display statistics."""
    
    print("="*60)
    print("Dataset Analysis")
    print("="*60)
    
    stats = {}
    
    for split in ['train', 'val', 'test']:
        split_path = data_path / split
        stats[split] = {}
        
        for class_dir in split_path.iterdir():
            if class_dir.is_dir():
                class_name = class_dir.name
                n_images = len(list(class_dir.glob('*')))
                stats[split][class_name] = n_images
    
    # Create DataFrame
    df = pd.DataFrame(stats).T
    df['Total'] = df.sum(axis=1)
    
    print("\nDataset Distribution:")
    print(df)
    
    total_images = df['Total'].sum()
    print(f"\nTotal Images: {total_images}")
    print(f"  Train: {df.loc['train', 'Total']} ({df.loc['train', 'Total']/total_images*100:.1f}%)")
    print(f"  Validation: {df.loc['val', 'Total']} ({df.loc['val', 'Total']/total_images*100:.1f}%)")
    print(f"  Test: {df.loc['test', 'Total']} ({df.loc['test', 'Total']/total_images*100:.1f}%)")
    
    return stats, df

stats, stats_df = analyze_dataset(DATA_PATH)

In [ ]:
def plot_dataset_distribution(stats, save_path):
    """Visualize dataset distribution."""
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    fig.suptitle('Leaf Disease Dataset Distribution', fontsize=14, fontweight='bold')
    
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']
    
    for idx, split in enumerate(['train', 'val', 'test']):
        ax = axes[idx]
        data = stats[split]
        
        classes = list(data.keys())
        counts = list(data.values())
        
        ax.bar(classes, counts, color=colors)
        ax.set_title(f'{split.upper()} Set ({sum(counts)} images)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Class')
        ax.set_ylabel('Number of Images')
        ax.tick_params(axis='x', rotation=45)
        
        # Add count labels on bars
        for i, (c, count) in enumerate(zip(classes, counts)):
            ax.text(i, count + max(counts)*0.02, str(count), 
                   ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_path / 'dataset_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_dataset_distribution(stats, MODEL_SAVE_PATH)

In [ ]:
def display_sample_images(data_path, save_path):
    """Display sample images from each class."""
    
    train_path = data_path / 'train'
    class_dirs = sorted([d for d in train_path.iterdir() if d.is_dir()])
    
    fig, axes = plt.subplots(len(class_dirs), 4, figsize=(14, 10))
    fig.suptitle('Sample Images from Each Class', fontsize=14, fontweight='bold')
    
    for row, cls_dir in enumerate(class_dirs):
        images = list(cls_dir.glob('*'))[:4]
        
        for col in range(4):
            ax = axes[row, col]
            if col < len(images):
                img = Image.open(images[col])
                ax.imshow(img)
                if col == 0:
                    ax.set_ylabel(cls_dir.name, fontsize=11, fontweight='bold')
            ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path / 'sample_images.png', dpi=150, bbox_inches='tight')
    plt.show()

display_sample_images(DATA_PATH, MODEL_SAVE_PATH)

## 4. Data Generators with Augmentation

**IMPORTANT:** Data augmentation is applied ONLY to training data to prevent data leaking.

In [ ]:
# Training Data Generator WITH Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Validation & Test Data Generator WITHOUT Augmentation (only rescaling)
val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

print("Data Generators Created:")
print("  Training: WITH augmentation (rotation, shift, flip, zoom)")
print("  Validation/Test: WITHOUT augmentation (only rescaling)")
print("\n  This prevents data leaking from augmented images.")

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    DATA_PATH / 'train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

validation_generator = val_test_datagen.flow_from_directory(
    DATA_PATH / 'val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    DATA_PATH / 'test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Verify class indices
print("\nClass Indices:")
print(train_generator.class_indices)

# Create reverse mapping
INDEX_TO_CLASS = {v: k for k, v in train_generator.class_indices.items()}
print("\nIndex to Class Mapping:")
print(INDEX_TO_CLASS)

## 5. Compute Class Weights for Imbalanced Data

In [ ]:
def compute_class_weights(generator):
    """Compute class weights to handle class imbalance."""
    
    # Get all labels
    labels = generator.classes
    
    # Compute class weights
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(labels),
        y=labels
    )
    
    class_weight_dict = dict(enumerate(class_weights))
    
    print("Class Weights (to handle imbalance):")
    for idx, weight in class_weight_dict.items():
        class_name = INDEX_TO_CLASS[idx]
        count = np.sum(labels == idx)
        print(f"  {class_name}: {weight:.4f} (n={count})")
    
    return class_weight_dict

class_weights = compute_class_weights(train_generator)

## 6. Define Focal Loss Function

Focal Loss helps with class imbalance by down-weighting easy examples and focusing on hard examples.

In [ ]:
def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss for multi-class classification.
    
    FL(p_t) = -alpha * (1 - p_t)^gamma * log(p_t)
    
    Args:
        gamma: Focusing parameter (default 2.0)
        alpha: Weighting factor (default 0.25)
    """
    def focal_loss_fn(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.keras.backend.clip(y_pred, epsilon, 1.0 - epsilon)
        
        cross_entropy = -y_true * tf.keras.backend.log(y_pred)
        focal_weight = tf.keras.backend.pow(1.0 - y_pred, gamma)
        focal_loss = alpha * focal_weight * cross_entropy
        
        return tf.keras.backend.sum(focal_loss, axis=-1)
    
    return focal_loss_fn

print(f"Focal Loss configured with gamma={FOCAL_GAMMA}, alpha={FOCAL_ALPHA}")
print("\nFocal Loss helps the model focus on hard-to-classify examples.")

## 7. Build Model Architecture

In [ ]:
def build_model(num_classes, trainable_base=False):
    """
    Build EfficientNetB0-based model for leaf disease classification.
    
    Args:
        num_classes: Number of output classes
        trainable_base: Whether base model is trainable
    
    Returns:
        model: Compiled Keras model
        base_model: Base EfficientNetB0 model
    """
    
    # Load pre-trained EfficientNetB0
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model layers
    base_model.trainable = trainable_base
    
    # Build model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Build model with frozen base
model, base_model = build_model(NUM_CLASSES, trainable_base=False)

# Display model summary
model.summary()

In [ ]:
# Count parameters
total_params = model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable_params = total_params - trainable_params

print(f"\nModel Parameters:")
print(f"  Total: {total_params:,}")
print(f"  Trainable: {trainable_params:,}")
print(f"  Non-trainable: {non_trainable_params:,}")

## 8. Compile Model and Setup Callbacks

In [ ]:
# Compile model for Phase 1 (frozen base)
model.compile(
    optimizer=optimizers.Adam(learning_rate=PHASE1_LR),
    loss=focal_loss(gamma=FOCAL_GAMMA, alpha=FOCAL_ALPHA),
    metrics=['accuracy']
)

print("Model compiled for Phase 1 (frozen base):")
print(f"  Optimizer: Adam (lr={PHASE1_LR})")
print(f"  Loss: Focal Loss (gamma={FOCAL_GAMMA}, alpha={FOCAL_ALPHA})")

In [ ]:
# Setup callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath=str(MODEL_SAVE_PATH / 'best_model_phase1.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured:")
print("  1. EarlyStopping (patience=8)")
print("  2. ReduceLROnPlateau (factor=0.5, patience=4)")
print("  3. ModelCheckpoint (save best model)")

## 9. Phase 1 Training - Frozen Base Layers

In [ ]:
print("="*60)
print("PHASE 1: Training with Frozen Base Layers")
print("="*60)
print(f"\nTraining for up to {PHASE1_EPOCHS} epochs...")
print(f"Base model (EfficientNetB0) is FROZEN - only training top layers.\n")

start_time = datetime.now()

history_phase1 = model.fit(
    train_generator,
    epochs=PHASE1_EPOCHS,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks_list,
    verbose=1
)

phase1_time = (datetime.now() - start_time).total_seconds() / 60
print(f"\nPhase 1 completed in {phase1_time:.2f} minutes")
print(f"Best validation accuracy: {max(history_phase1.history['val_accuracy']):.4f}")

## 10. Phase 2 Training - Fine-tuning

In [ ]:
print("="*60)
print("PHASE 2: Fine-tuning with Unfrozen Base Layers")
print("="*60)

# Unfreeze base model
base_model.trainable = True

# Freeze early layers, unfreeze later layers
for layer in base_model.layers[:-30]:
    layer.trainable = False

trainable_layers = sum([1 for layer in base_model.layers if layer.trainable])
print(f"\nUnfrozen {trainable_layers} layers in base model for fine-tuning.")

# Recompile with lower learning rate
model.compile(
    optimizer=optimizers.Adam(learning_rate=PHASE2_LR),
    loss=focal_loss(gamma=FOCAL_GAMMA, alpha=FOCAL_ALPHA),
    metrics=['accuracy']
)

print(f"Recompiled with learning rate: {PHASE2_LR}")

In [ ]:
# Update checkpoint path for phase 2
callbacks_list_phase2 = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-8,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath=str(MODEL_SAVE_PATH / 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print(f"\nTraining for up to {PHASE2_EPOCHS} epochs...\n")

start_time = datetime.now()

history_phase2 = model.fit(
    train_generator,
    epochs=PHASE2_EPOCHS,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks_list_phase2,
    verbose=1
)

phase2_time = (datetime.now() - start_time).total_seconds() / 60
print(f"\nPhase 2 completed in {phase2_time:.2f} minutes")
print(f"Best validation accuracy: {max(history_phase2.history['val_accuracy']):.4f}")

## 11. Training History Visualization

In [ ]:
def plot_training_history(history1, history2, save_path):
    """Plot combined training history for both phases."""
    
    # Combine histories
    acc = history1.history['accuracy'] + history2.history['accuracy']
    val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    phase1_end = len(history1.history['accuracy'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle('Training History - Leaf Disease Detection Model', 
                 fontsize=14, fontweight='bold')
    
    # Accuracy plot
    ax1 = axes[0]
    ax1.plot(epochs, acc, 'b-', label='Training Accuracy', linewidth=2)
    ax1.plot(epochs, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
    ax1.axvline(x=phase1_end, color='g', linestyle='--', label='Phase 1 → Phase 2')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.legend(loc='lower right')
    ax1.grid(True, alpha=0.3)
    
    # Loss plot
    ax2 = axes[1]
    ax2.plot(epochs, loss, 'b-', label='Training Loss', linewidth=2)
    ax2.plot(epochs, val_loss, 'r-', label='Validation Loss', linewidth=2)
    ax2.axvline(x=phase1_end, color='g', linestyle='--', label='Phase 1 → Phase 2')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Model Loss')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path / 'training_history.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_training_history(history_phase1, history_phase2, MODEL_SAVE_PATH)

## 12. Model Evaluation on Test Set

In [ ]:
print("="*60)
print("Model Evaluation on Test Set")
print("="*60)

# Load best model
best_model = tf.keras.models.load_model(
    MODEL_SAVE_PATH / 'best_model.keras',
    custom_objects={'focal_loss_fn': focal_loss(gamma=FOCAL_GAMMA, alpha=FOCAL_ALPHA)}
)

# Evaluate on test set
test_loss, test_accuracy = best_model.evaluate(test_generator, verbose=1)

print(f"\nTest Results:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

## 13. Detailed Classification Report

In [ ]:
# Get predictions
test_generator.reset()
predictions = best_model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Generate classification report
print("\n" + "="*60)
print("Classification Report (Class-wise Metrics)")
print("="*60)

# Get class names in correct order
class_names_ordered = [k for k, v in sorted(test_generator.class_indices.items(), key=lambda x: x[1])]

report = classification_report(y_true, y_pred, target_names=class_names_ordered, digits=4)
print(report)

In [ ]:
# Get detailed metrics
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

# Create metrics DataFrame
metrics_df = pd.DataFrame({
    'Class': class_names_ordered,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\n" + "="*60)
print("Class-wise Metrics Summary")
print("="*60)
print(metrics_df.to_string(index=False))

# Calculate macro averages
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = np.mean(f1)

print(f"\n{'='*60}")
print("Overall Metrics")
print("="*60)
print(f"  Accuracy:        {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  Macro Precision: {macro_precision:.4f}")
print(f"  Macro Recall:    {macro_recall:.4f}")
print(f"  Macro F1-Score:  {macro_f1:.4f}")

## 14. Confusion Matrix Visualization

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names, save_path):
    """Plot confusion matrix with percentages."""
    
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle('Confusion Matrix - Leaf Disease Detection Model', 
                 fontsize=14, fontweight='bold')
    
    # Raw counts
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names, ax=axes[0])
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].set_title('Counts')
    
    # Percentages
    sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names, ax=axes[1])
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')
    axes[1].set_title('Percentages (%)')
    
    plt.tight_layout()
    plt.savefig(save_path / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(y_true, y_pred, class_names_ordered, MODEL_SAVE_PATH)

## 15. Per-Class Metrics Visualization

In [ ]:
def plot_per_class_metrics(metrics_df, save_path):
    """Visualize precision, recall, and F1-score per class."""
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(metrics_df))
    width = 0.25
    
    bars1 = ax.bar(x - width, metrics_df['Precision'], width, label='Precision', color='#3498db')
    bars2 = ax.bar(x, metrics_df['Recall'], width, label='Recall', color='#2ecc71')
    bars3 = ax.bar(x + width, metrics_df['F1-Score'], width, label='F1-Score', color='#e74c3c')
    
    ax.set_xlabel('Class', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Per-Class Metrics: Precision, Recall, F1-Score', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_df['Class'])
    ax.legend(loc='lower right')
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.savefig(save_path / 'per_class_metrics.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_per_class_metrics(metrics_df, MODEL_SAVE_PATH)

## 16. Sample Predictions Visualization

In [ ]:
def plot_sample_predictions(generator, model, save_path, n_samples=12):
    """Display sample predictions with confidence scores."""
    
    generator.reset()
    
    # Get a batch
    images, labels = next(generator)
    predictions = model.predict(images, verbose=0)
    
    # Select samples
    n_samples = min(n_samples, len(images))
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    fig.suptitle('Sample Predictions with Confidence Scores', fontsize=14, fontweight='bold')
    
    class_names = list(generator.class_indices.keys())
    
    for idx in range(n_samples):
        ax = axes[idx // 4, idx % 4]
        
        # Display image
        ax.imshow(images[idx])
        
        # Get predictions
        pred_idx = np.argmax(predictions[idx])
        pred_class = class_names[pred_idx]
        pred_conf = predictions[idx][pred_idx]
        
        true_idx = np.argmax(labels[idx])
        true_class = class_names[true_idx]
        
        # Set title color based on correctness
        color = 'green' if pred_idx == true_idx else 'red'
        
        ax.set_title(f'True: {true_class}\nPred: {pred_class} ({pred_conf:.2%})', 
                    fontsize=10, color=color, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path / 'sample_predictions.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_sample_predictions(test_generator, best_model, MODEL_SAVE_PATH)

## 17. Save Model Information

In [ ]:
# Calculate total training time
total_time = phase1_time + phase2_time
total_epochs = len(history_phase1.history['accuracy']) + len(history_phase2.history['accuracy'])

# Model information
model_info = {
    'model_name': MODEL_NAME,
    'version': '1.0',
    'architecture': 'EfficientNetB0',
    'framework': 'TensorFlow/Keras',
    'tensorflow_version': tf.__version__,
    'image_size': IMG_SIZE,
    'num_classes': NUM_CLASSES,
    'class_names': class_names_ordered,
    'class_indices': test_generator.class_indices,
    'training': {
        'total_epochs': total_epochs,
        'phase1_epochs': len(history_phase1.history['accuracy']),
        'phase2_epochs': len(history_phase2.history['accuracy']),
        'phase1_lr': PHASE1_LR,
        'phase2_lr': PHASE2_LR,
        'batch_size': BATCH_SIZE,
        'focal_gamma': FOCAL_GAMMA,
        'focal_alpha': FOCAL_ALPHA,
        'total_time_minutes': round(total_time, 2),
        'optimizer': 'Adam',
        'loss_function': 'Focal Loss'
    },
    'performance': {
        'test_accuracy': float(test_accuracy),
        'test_loss': float(test_loss),
        'macro_precision': float(macro_precision),
        'macro_recall': float(macro_recall),
        'macro_f1': float(macro_f1)
    },
    'per_class_metrics': {
        class_names_ordered[i]: {
            'precision': float(precision[i]),
            'recall': float(recall[i]),
            'f1_score': float(f1[i]),
            'support': int(support[i])
        }
        for i in range(len(class_names_ordered))
    },
    'dataset': {
        'train_samples': train_generator.samples,
        'val_samples': validation_generator.samples,
        'test_samples': test_generator.samples,
        'total_samples': train_generator.samples + validation_generator.samples + test_generator.samples
    },
    'augmentation': {
        'rotation_range': 30,
        'width_shift_range': 0.2,
        'height_shift_range': 0.2,
        'shear_range': 0.2,
        'zoom_range': 0.2,
        'horizontal_flip': True,
        'vertical_flip': True
    },
    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Save model info
with open(MODEL_SAVE_PATH / 'model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"Model information saved to: {MODEL_SAVE_PATH / 'model_info.json'}")

## 18. Final Summary

In [ ]:
print("\n" + "="*70)
print("           TRAINING COMPLETE - FINAL SUMMARY")
print("="*70)

print(f"\n📊 Model: {MODEL_NAME}")
print(f"   Architecture: EfficientNetB0 (Transfer Learning)")
print(f"   Classes: {', '.join(class_names_ordered)}")

print(f"\n⏱️  Training Time: {total_time:.2f} minutes ({total_epochs} epochs)")
print(f"   Phase 1: {len(history_phase1.history['accuracy'])} epochs ({phase1_time:.2f} min)")
print(f"   Phase 2: {len(history_phase2.history['accuracy'])} epochs ({phase2_time:.2f} min)")

print(f"\n📈 Test Results:")
print(f"   Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   Precision: {macro_precision:.4f}")
print(f"   Recall:    {macro_recall:.4f}")
print(f"   F1-Score:  {macro_f1:.4f}")

print(f"\n📋 Per-Class Performance:")
for i, cls in enumerate(class_names_ordered):
    print(f"   {cls:15s} - P: {precision[i]:.4f}, R: {recall[i]:.4f}, F1: {f1[i]:.4f}")

print(f"\n📁 Saved Files:")
print(f"   - best_model.keras")
print(f"   - best_model_phase1.keras")
print(f"   - model_info.json")
print(f"   - dataset_distribution.png")
print(f"   - sample_images.png")
print(f"   - training_history.png")
print(f"   - confusion_matrix.png")
print(f"   - per_class_metrics.png")
print(f"   - sample_predictions.png")

print("\n" + "="*70)
print("     Model ready for deployment in Flask API!")
print("="*70)